In [14]:
import requests
from datetime import datetime
from datetime import date
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

cities_list = pd.read_excel('cities_list.xlsx')

API_KEY = os.getenv('API_KEY')

def get_air_pollution_Data(API_KEY, lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error requisition: {response.status_code}')
        return None

In [2]:
cities_list

,name,lon,lat,country
0,Abidjan,-4.00167,5.35444,Ivory Coast
1,Abu Dhabi,54.39696,24.45118,United Arab Emirates
2,Abuja,7.49508,9.05785,Nigeria
3,Accra,-0.19690,5.55602,Ghana
4,Addis Ababa,38.74689,9.02497,Ethiopia
...,...,...,...,...
366,Yaounde,11.51667,3.86667,Cameroon
367,Yekaterinburg,60.61220,56.85190,Russia
368,Yerevan,44.51361,40.18111,Armenia
369,Yokohama,139.65000,35.43333,Japan


In [3]:
get_air_pollution_Data(API_KEY, -4.00167, 5.35444)

{'coord': {'lon': 5.3544, 'lat': -4.0017},
 'list': [{'main': {'aqi': 2},
   'components': {'co': 76.08,
    'no': 0,
    'no2': 0.03,
    'o3': 46.93,
    'so2': 0.23,
    'pm2_5': 12.12,
    'pm10': 38.26,
    'nh3': 0},
   'dt': 1748628000},
  {'main': {'aqi': 2},
   'components': {'co': 75.83,
    'no': 0,
    'no2': 0.03,
    'o3': 47.52,
    'so2': 0.23,
    'pm2_5': 12.01,
    'pm10': 38.38,
    'nh3': 0},
   'dt': 1748631600},
  {'main': {'aqi': 2},
   'components': {'co': 75.66,
    'no': 0,
    'no2': 0.03,
    'o3': 48.2,
    'so2': 0.23,
    'pm2_5': 11.81,
    'pm10': 38.01,
    'nh3': 0},
   'dt': 1748635200},
  {'main': {'aqi': 2},
   'components': {'co': 75.58,
    'no': 0,
    'no2': 0.03,
    'o3': 48.7,
    'so2': 0.23,
    'pm2_5': 11.6,
    'pm10': 37.62,
    'nh3': 0},
   'dt': 1748638800},
  {'main': {'aqi': 2},
   'components': {'co': 75.3,
    'no': 0,
    'no2': 0.03,
    'o3': 48.98,
    'so2': 0.23,
    'pm2_5': 11.4,
    'pm10': 37.25,
    'nh3': 0},
   'dt

In [4]:
def process_pollution_data(data, city, country):
    if not data or 'list'not in data:
        return None
    
    formatted_data = []
    for item in data['list']:
        row = {
            'dt': item['dt'],
            'aqi': item['main']['aqi'],
            'city': city,
            'country': country
        }
        if isinstance(item.get('components'), dict):
            row.update(item['components'])
        formatted_data.append(row)

    coord = data.get('coord', {})
    if coord:
        row = {
            'lon': coord.get('lon', None),
            'lat': coord.get('lat', None),
            'city': city,  # Keep city and country for consistency
            'country': country
        }
        formatted_data.append(row)

    return formatted_data

In [5]:
all_data = []

In [6]:
for index, row in cities_list.iterrows():
    lon = row.lon
    lat = row.lat
    country = row.country
    city = row.name

    data = get_air_pollution_Data(API_KEY, lat, lon)

    if data:
        city_data = process_pollution_data(data, city, country)
        if city_data:
            all_data.append(city_data)

In [7]:
df = pd.DataFrame(all_data)

In [8]:
columns = ['dt', 'city', 'country', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']

In [9]:
correction_data = [item for sublist in all_data for item in sublist]
df = pd.DataFrame(correction_data)

In [ ]:
# datetime.datetime(1748970000)
date.fromtimestamp(1748970000)

datetime.date(2025, 6, 3)

In [10]:
df

,dt,aqi,city,country,co,no,no2,o3,so2,pm2_5,pm10,nh3,lon,lat
0,1.748628e+09,1.0,0,Ivory Coast,80.58,0.01,0.21,36.46,0.26,3.38,10.72,0.00,NaN,NaN
1,1.748632e+09,1.0,0,Ivory Coast,81.43,0.00,0.25,35.92,0.28,3.46,10.38,0.00,NaN,NaN
2,1.748635e+09,1.0,0,Ivory Coast,82.72,0.00,0.26,35.56,0.27,3.43,9.32,0.00,NaN,NaN
3,1.748639e+09,1.0,0,Ivory Coast,84.28,0.00,0.26,35.89,0.27,3.31,8.14,0.00,NaN,NaN
4,1.748642e+09,1.0,0,Ivory Coast,84.68,0.00,0.20,36.52,0.20,2.86,5.96,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35982,1.748959e+09,2.0,370,Croatia,93.72,0.07,0.36,90.80,1.18,12.84,15.38,0.53,NaN,NaN
35983,1.748963e+09,2.0,370,Croatia,93.72,0.08,0.43,90.21,1.44,16.23,19.01,0.49,NaN,NaN
35984,1.748966e+09,2.0,370,Croatia,94.39,0.19,1.02,82.85,1.69,21.08,23.95,0.43,NaN,NaN
35985,1.748970e+09,3.0,370,Croatia,95.60,0.30,1.94,74.24,2.00,27.15,30.07,0.33,NaN,NaN
